In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [3]:
import keras_tuner as kt
hp = kt.HyperParameters()
import numpy as np
import tensorflow as tf
import pandas as pd
import shutil
import seaborn as sns
import sklearn, os, time, h5py
import SimpleITK as sitk
import matplotlib.pyplot as plt
from random import randint
import random
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from math import floor
from tensorflow.keras.applications import InceptionV3, DenseNet121
from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings('ignore')

In [4]:
root_path = '/data1/registration/anhi/d2/'
# import zipfile
# with zipfile.ZipFile(root_path + "Dataset.zip","r") as zip_ref:
#     zip_ref.extractall(root_path + "")

datasetFolderName = root_path + 'Dataset1'
sourceFiles = []
classLabels = ['light', 'moderate', 'no', 'severe']
X = []
Y = []

img_rows, img_cols = 224, 224 
train_path = datasetFolderName+'/train/'
validation_path = datasetFolderName+'/valid/'
test_path = datasetFolderName+'/test/'

def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles = os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers = int(len(sourceFiles)*splitRate)
        transferIndex = random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', splitRate)
                               
def my_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1Score = f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("recall : {}".format(recall))
    print("f1Score : {}".format(f1Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, recall, f1Score

def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
            if(folderName==classLabels[i]):
                Y.append(i)
for i in range(len(classLabels)):
    prepareNameWithLabels(classLabels[i])
    
X = np.asarray(X)
Y = np.asarray(Y)
IMAGE_SIZE = [224, 224]

In [5]:
def CNN_model(hp):
    mobilenet = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

    for layer in mobilenet.layers:
        layer.trainable = False
    x = Flatten()(mobilenet.output)

    for i in range(hp.Int('layers', 1, 2)):
        x = Dense(units=hp.Int('units_' + str(i), 64, 512, step=64),
                  activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid', 'elu', 'selu', 'tanh']))(x)
        x = Dropout(rate=hp.Float('dropout_' + str(i), 0, 0.4, step=0.1))(x)

    prediction = Dense(len(classLabels), activation='softmax')(x)
    model = Model(inputs=mobilenet.input, outputs=prediction)

    hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    if hp_optimizer == 'sgd':
        optimizer = SGD(learning_rate=hp_lr)
    elif hp_optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=hp_lr)
    elif hp_optimizer == 'adam':
        optimizer = Adam(learning_rate=hp_lr)
    else:
        raise
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])       
    return model

tuner = kt.BayesianOptimization(CNN_model,
                                objective='val_accuracy',max_trials=30,project_name="BayesianOptimization")

2022-09-30 21:37:46.524781: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 21:37:47.331163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10083 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:18:00.0, compute capability: 8.6


In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path,target_size=(img_rows, img_cols),       
    batch_size=32, class_mode='categorical', subset='training')     

validation_generator = validation_datagen.flow_from_directory(validation_path,  target_size=(img_rows, img_cols),      
    batch_size=32, class_mode='categorical')   

test_generator = test_datagen.flow_from_directory(test_path, target_size=(img_rows, img_cols),       
    batch_size=32,  class_mode=None, shuffle=False) 


tuner.search(train_generator, epochs=30, validation_data=validation_generator)
#              , callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=3)])

Trial 30 Complete [00h 11m 31s]
val_accuracy: 0.6725000143051147

Best val_accuracy So Far: 0.7049999833106995
Total elapsed time: 06h 08m 47s
INFO:tensorflow:Oracle triggered exit


In [1]:
# # HyperBand algorithm from keras tuner
# tuner = kt.Hyperband(
#     CNN_model,
#     objective=kt.Objective('accuracy',direction='max'),
#     max_epochs=10,
# #     directory='keras_tuner_dir',
#     project_name='keras_tuner_demo')


In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'elu', 'selu', 'tanh'], 'ordered': False}
dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.4, 'step': 0.1, 'sampling': None}
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'rmsprop', 'adam'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'elu', 'selu', 'tanh'], 'ordered': False}
dropout_1 (Float)
{

In [8]:
tuner.results_summary()

Results summary
Results in ./BayesianOptimization
Showing 10 best trials
Trial summary
Hyperparameters:
layers: 2
units_0: 192
act_0: sigmoid
dropout_0: 0.4
optimizer: rmsprop
learning_rate: 0.0001
units_1: 448
act_1: relu
dropout_1: 0.1
Score: 0.6899999976158142
Trial summary
Hyperparameters:
layers: 1
units_0: 512
act_0: tanh
dropout_0: 0.4
optimizer: adam
learning_rate: 0.0001
units_1: 512
act_1: relu
dropout_1: 0.0
Score: 0.6899999976158142
Trial summary
Hyperparameters:
layers: 2
units_0: 64
act_0: relu
dropout_0: 0.1
optimizer: rmsprop
learning_rate: 0.001
units_1: 64
act_1: relu
dropout_1: 0.0
Score: 0.6875
Trial summary
Hyperparameters:
layers: 1
units_0: 64
act_0: relu
dropout_0: 0.0
optimizer: adam
learning_rate: 0.0001
units_1: 64
act_1: relu
dropout_1: 0.0
Score: 0.6850000023841858
Trial summary
Hyperparameters:
layers: 2
units_0: 256
act_0: tanh
dropout_0: 0.4
optimizer: adam
learning_rate: 0.0001
units_1: 512
act_1: tanh
dropout_1: 0.0
Score: 0.6825000047683716
Trial summ

In [9]:
best_hp = tuner.get_best_hyperparameters(1)[0]
print(best_hp.values)

{'layers': 2, 'units_0': 192, 'act_0': 'sigmoid', 'dropout_0': 0.4, 'optimizer': 'rmsprop', 'learning_rate': 0.0001, 'units_1': 448, 'act_1': 'relu', 'dropout_1': 0.1}
